In [2]:
import pandas as pd
import numpy as np

Load Data

# NHANES II

In [84]:
nhanesII = pd.read_stata("/Users/bryankim/Documents/NBER/Case Deaton/Data/FRS/Old Framingham/NHANESII/all_nhanesnhanesII_for_frs.dta")
nhanesII["sex"].replace("Male", 1,inplace=True)
nhanesII["sex"].replace("Female", 2,inplace=True)

In [86]:
nhanesII[["sex","treated","wgt"]].head().dtypes
nhanesII["sex"].head()
nhanesII["sex"].astype('float32').dtypes
nhanesII['sex'] = pd.factorize(nhanesII['sex'])[0] + 1

See Ranges

In [39]:
print(nhanesII[["age","sex","hdl","lbxtc","treated","asbp","smq020","lbxgh"]].min())
print(nhanesII[["AGEYR_AT_INT","SEX","lbdhdd","lbxtc","treated","asbp","curr_smq"]].max())
nhanesII.curr_smq.value_counts()
nhanesII["curr_smq"][1]==1

AGEYR_AT_INT    45.0
SEX              1.0
lbdhdd          15.0
lbxtc           80.0
treated          0.0
asbp            70.0
curr_smq         0.0
dtype: float64
AGEYR_AT_INT     74.0
SEX               2.0
lbdhdd          187.0
lbxtc           644.0
treated           1.0
asbp            300.0
curr_smq          1.0
dtype: float64


False

Create `treated` variable

Define `check` functions for females

In [5]:
def ftchol_pts(totchol,age):
    if totchol < 160:
        return(0)
    if totchol in range(160,199+1):
        if age in range(20,39+1): return(4)
        if age in range(40,49+1): return(3) 
        if age in range(50,59+1): return(2) 
        if age in range(60,69+1): return(1) 
        if age >= 70: return(1) 
    if totchol in range(200,239+1):
        if age in range(20,39+1): return(8) 
        if age in range(40,49+1): return(6) 
        if age in range(50,59+1): return(4) 
        if age in range(60,69+1): return(2) 
        if age >= 70: return(1) 
    if totchol in range(240,279+1):
        if age in range(20,39+1): return(11) 
        if age in range(40,49+1): return(8) 
        if age in range(50,59+1): return(5) 
        if age in range(60,69+1): return(3) 
        if age >= 70: return(2) 
    if totchol >= 280:
        if age in range(20,39+1): return(13) 
        if age in range(40,49+1): return(10)
        if age in range(50,59+1): return(7)
        if age in range(60,69+1): return(4) 
        if age >= 70: return(2) 
    
def fsmoke_pts(curr_smq,age):
    if curr_smq == 0:
        return(0)
    if curr_smq == 1:
        if age in range(20,39+1): return(9) 
        if age in range(40,49+1): return(7) 
        if age in range(50,59+1): return(4) 
        if age in range(60,69+1): return(2)
        if age >= 70: return(1) 

    
def fsbp_pts(sbp,treated):
    if sbp < 120:
        return(0)
    if sbp in range(120,129+1):
        return(3) if treated == 1 else 1
    if sbp in range(130,139+1):
        return(4) if treated == 1 else 2
    if sbp in range(140,159+1):
        return(5) if treated == 1 else 3
    if sbp >= 160:
        return(6) if treated == 1 else 4
    
def fage_pts(age):
    if age in range(45,49+1):
        return(3)
    if age in range(50,54+1):
        return(6)
    if age in range(55,59+1):
        return(8)
    if age in range(60,64+1):
        return(10)
    if age in range(65,69+1):
        return(12)
    if age in range(70,74+1):
        return(14)
    if age >= 75:
        return(16)

def fhdl_pts(hdl):
    if hdl >= 60:
        return(-1)
    if hdl in range(50,59+1):
        return(0)
    if hdl in range(40,49+1):
        return(1)
    if hdl < 40:
        return(2)

Define `check` functions for males

In [6]:
def mtchol_pts(totchol,age):
    if totchol < 160:
        return(0)
    if totchol in range(160,199+1):
        if age in range(20,39+1): return(4)
        if age in range(40,49+1): return(3) 
        if age in range(50,59+1): return(2) 
        if age in range(60,69+1): return(1) 
        if age >= 70: return(0) 
    if totchol in range(200,239+1):
        if age in range(20,39+1): return(7) 
        if age in range(40,49+1): return(5) 
        if age in range(50,59+1): return(3) 
        if age in range(60,69+1): return(1) 
        if age >= 70: return(0) 
    if totchol in range(240,279+1):
        if age in range(20,39+1): return(9) 
        if age in range(40,49+1): return(6) 
        if age in range(50,59+1): return(4) 
        if age in range(60,69+1): return(2) 
        if age >= 70: return(1) 
    if totchol >= 280:
        if age in range(20,39+1): return(11) 
        if age in range(40,49+1): return(8)
        if age in range(50,59+1): return(5)
        if age in range(60,69+1): return(3) 
        if age >= 70:return(1) 


#
 
def msmoke_pts(curr_smq,age):
    if curr_smq == 0:
        return(0)
    if curr_smq == 1:
        if age in range(20,39+1): return(8) 
        if age in range(40,49+1): return(5) 
        if age in range(50,59+1): return(3) 
        if age in range(60,69+1): return(1)
        if age >= 70: return(1)
    
def msbp_pts(sbp,treated):
    if sbp < 120:
        return(0)
    if sbp in range(120,129+1):
        return(1) if treated == 1 else 0
    if sbp in range(130,159+1):
        return(2) if treated == 1 else 1
    if sbp >= 160:
        return(3) if treated == 1 else 2
    
def mage_pts(age):
    if age in range(45,49+1):
        return(3)
    if age in range(50,54+1):
        return(6)
    if age in range(55,59+1):
        return(8)
    if age in range(60,64+1):
        return(10)
    if age in range(65,69+1):
        return(11)
    if age in range(70,74+1):
        return(12)
    if age >= 75:
        return(13)

def mhdl_pts(hdl):
    if hdl >= 60:
        return(-1)
    if hdl in range(50,59+1):
        return(0)
    if hdl in range(40,49+1):
        return(1)
    if hdl < 40:
        return(2)

In [31]:
def get_frs_atp3(sex,age,hdl,totchol,sbp,curr_smq,treated):
    risk_score = 0
    if sex == 1:
        risk_score += mage_pts(age)
        risk_score += mhdl_pts(hdl)
        risk_score += mtchol_pts(totchol,age)
        risk_score += msbp_pts(sbp,treated)
        risk_score += msmoke_pts(curr_smq,age)
    if sex == 2:
        risk_score += fage_pts(age)
        risk_score += fhdl_pts(hdl)
        risk_score += ftchol_pts(totchol,age)
        risk_score += fsbp_pts(sbp,treated)
        risk_score += fsmoke_pts(curr_smq,age)
    return(risk_score)

def get_frs_atp3_nhanesII(x):
    risk_score = 0
    if x["sex"] == 1:
        risk_score += mage_pts(x["age"])
        risk_score += mhdl_pts(x["hdl"])
        risk_score += mtchol_pts(x["lbxtc"],x["age"])
        risk_score += msbp_pts(x["asbp"],x["treated"])
        risk_score += msmoke_pts(x["curr_smq"],x["age"])
    if x["sex"] == 2:
        risk_score += fage_pts(x["AGEYR_AT_INT"])
        risk_score += fhdl_pts(x["hdl"])
        risk_score += ftchol_pts(x["lbxtc"],x["age"])
        risk_score += fsbp_pts(x["asbp"],x["treated"])
        risk_score += fsmoke_pts(x["curr_smq"],x["age"])
    return(risk_score)


    Need to complete cases of all variable (fields) before running `get_frs` function

In [35]:
from sklearn.datasets import load_iris
data = load_iris()
features = pd.DataFrame(data = data['data'], columns= data ['feature_names'])
features.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [131]:
nhanesII=nhanesII.dropna(subset=["age","sex","hdl","lbxtc","asbp","smq020","treated"])
# count number of rows
nhanesII.sex.count()

5328

In [64]:
nhanesII[["age","sex","hdl","lbxtc","treated","asbp","smq020","treated"]].dtypes

age        float32
sex          int64
hdl        float32
lbxtc      float64
treated    float32
asbp       float32
smq020     float64
treated    float32
dtype: object

In [69]:
nhanesII["frs_atp3"]=list(map(get_frs_atp3,nhanesII["sex"],nhanesII["age"],nhanesII["hdl"],nhanesII["lbxtc"],nhanesII["asbp"],nhanesII["curr_smq"],nhanesII["treated"])) 
nhanesII["frs_atp3"].head()

287    15
288    18
289    15
290    19
291    17
Name: frs_atp3, dtype: int64

In [71]:
nhanesII.to_stata("/Users/bryankim/Documents/NBER/Case Deaton/Data/FRS/ATP3/nhanesII_frs_atp3.dta")


# NHANES III 

In [114]:
def get_frs_atp3_nhanesIII(x):
    risk_score = 0
    if x["sex"] == 1:
        risk_score += mage_pts(x["age"])
        risk_score += mhdl_pts(x["hdl"])
        risk_score += mtchol_pts(x["lbxtc"],x["age"])
        risk_score += msbp_pts(x["asbp"],x["treated_hpt"])
        risk_score += msmoke_pts(x["curr_smq"],x["age"])
    if x["sex"] == 2:
        risk_score += fage_pts(x["age"])
        risk_score += fhdl_pts(x["hdl"])
        risk_score += ftchol_pts(x["lbxtc"],x["HSAGEIR"])
        risk_score += fsbp_pts(x["asbp"],x["treated_hpt"])
        risk_score += fsmoke_pts(x["curr_smq"],x["age"])
    return(risk_score)

## Phase 1

In [159]:
nhanesIIIP1 = pd.read_stata("/Users/bryankim/Documents/NBER/Case Deaton/Data/FRS/Old Framingham/NHANESIII/all_nhanesnhanesIIIP1_for_frs.dta")
nhanesIIIP1[["age","sex","hdl","lbxtc","asbp","HAR1","treated_hpt"]].head()

,age,sex,hdl,lbxtc,asbp,HAR1,treated_hpt
0,48.0,Female,66.0,236.0,131.0,1,0.0
1,48.0,Male,51.0,260.0,120.0,1,0.0
2,67.0,Female,NaN,NaN,NaN,2,0.0
3,56.0,Female,42.0,156.0,128.0,1,0.0
4,50.0,Male,47.0,244.0,117.0,2,0.0


In [160]:
nhanesIIIP1=nhanesIIIP1.dropna(subset=["age","sex","hdl","lbxtc","asbp","HAR1","treated_hpt"])
# count number of rows
nhanesIIIP1.sex.count()

2988

In [161]:
print(nhanesIIIP1[["sex","age","hdl","lbxtc","treated_hpt","asbp","curr_smq"]].min())
print(nhanesIIIP1[["sex","age","hdl","lbxtc","treated_hpt","asbp","curr_smq"]].max())
nhanesIIIP1.curr_smq.value_counts()
nhanesIIIP1["curr_smq"][1]==1

sex            Female
age                45
hdl                13
lbxtc             103
treated_hpt         0
asbp               80
curr_smq            0
dtype: object
sex            Male
age              74
hdl             191
lbxtc           464
treated_hpt       1
asbp            230
curr_smq          1
dtype: object


False

In [162]:
nhanesIIIP1[["sex","treated","wgt"]].head().dtypes

sex        category
treated     float32
wgt         float32
dtype: object

In [163]:
nhanesIIIP1["sex"].head()

0    Female
1      Male
3    Female
4      Male
5      Male
Name: sex, dtype: category
Categories (2, object): [Male < Female]

In [164]:
nhanesIIIP1["sex"].replace("Male", 1,inplace=True)

In [165]:
nhanesIIIP1["sex"].replace("Female", 2,inplace=True)

In [166]:
nhanesIIIP1['sex'] = pd.factorize(nhanesIIIP1['sex'])[0] + 1

In [167]:
nhanesIIIP1[["sex","age","lbxtc","asbp","curr_smq","treated_hpt","wgt","hdl"]].head().dtypes

sex              int64
age            float32
lbxtc          float64
asbp           float32
curr_smq       float32
treated_hpt    float32
wgt            float32
hdl            float32
dtype: object

In [168]:
nhanesIIIP1.hdl.unique()

array([ 66.,  51.,  42.,  47.,  30.,  53.,  46.,  36.,  62.,  44.,  35.,
        78.,  54.,  45.,  33.,  37.,  60.,  58.,  41.,  52.,  65.,  83.,
        38.,  34.,  49.,  48.,  43.,  81.,  56.,  61.,  87.,  69.,  50.,
        68.,  40.,  26.,  91.,  55.,  98.,  72.,  64.,  71.,  22.,  99.,
       104.,  25.,  79.,  14.,  39.,  28.,  63.,  76.,  32.,  73.,  59.,
        77.,  29.,  92., 106.,  24.,  31.,  70.,  57.,  80.,  84.,  75.,
        74.,  67.,  86., 100.,  23.,  93., 110.,  85., 103.,  88.,  19.,
       101.,  13.,  27.,  82.,  21., 112.,  97.,  95., 105., 120.,  20.,
       117., 123.,  94., 113.,  96.,  89.,  90., 121., 136., 118., 122.,
       107., 191., 109., 124.], dtype=float32)

In [169]:
nhanesIIIP1["frs_atp3"]=list(map(get_frs_atp3,nhanesIIIP1["sex"],nhanesIIIP1["age"],nhanesIIIP1["hdl"],nhanesIIIP1["lbxtc"],nhanesIIIP1["asbp"],nhanesIIIP1["curr_smq"],nhanesIIIP1["treated_hpt"])) 

In [170]:
nhanesIIIP1["frs_atp3"].head()

0     8
1    12
3     9
4    12
5    21
Name: frs_atp3, dtype: int64

save

In [171]:
nhanesIIIP1.to_stata("/Users/bryankim/Documents/NBER/Case Deaton/Data/FRS/ATP3/nhanesIII_P1x_frs_atp3.dta")

## Phase 2

In [172]:
nhanesIIIP2 = pd.read_stata("/Users/bryankim/Documents/NBER/Case Deaton/Data/FRS/Old Framingham/NHANESIII/all_nhanesnhanesIIIP2_for_frs.dta")
nhanesIIIP2[["age","sex","hdl","lbxtc","asbp","HAR1","treated_hpt"]].head()

,age,sex,hdl,lbxtc,asbp,HAR1,treated_hpt
0,60.0,Female,49.0,207.0,131.0,1,1.0
1,47.0,Female,56.0,261.0,157.0,2,1.0
2,58.0,Male,32.0,278.0,125.0,1,0.0
3,46.0,Female,57.0,166.0,121.0,2,0.0
4,73.0,Female,40.0,272.0,162.0,2,0.0


In [173]:
nhanesIIIP2=nhanesIIIP2.dropna(subset=["age","sex","hdl","lbxtc","asbp","HAR1","treated_hpt"])
# count number of rows
nhanesIIIP2.sex.count()

2966

In [174]:
nhanesIIIP2["sex"].replace("Male", 1,inplace=True)
nhanesIIIP2["sex"].replace("Female", 2,inplace=True)
nhanesIIIP2['sex'] = pd.factorize(nhanesIIIP2['sex'])[0] + 1
nhanesIIIP2[["sex","age","lbxtc","asbp","curr_smq","treated_hpt","wgt","hdl"]].head().dtypes

sex              int64
age            float32
lbxtc          float64
asbp           float32
curr_smq       float32
treated_hpt    float32
wgt            float32
hdl            float32
dtype: object

In [175]:
nhanesIIIP2["frs_atp3"]=list(map(get_frs_atp3,nhanesIIIP2["sex"],nhanesIIIP2["age"],nhanesIIIP2["hdl"],nhanesIIIP2["lbxtc"],nhanesIIIP2["asbp"],nhanesIIIP2["curr_smq"],nhanesIIIP2["treated_hpt"])) 

In [176]:
nhanesIIIP2["frs_atp3"].head()

0    14
1    11
2    16
3     6
4    16
Name: frs_atp3, dtype: int64

save

In [158]:
nhanesIIIP2.to_stata("/Users/bryankim/Documents/NBER/Case Deaton/Data/FRS/ATP3/nhanesIII_P2x_frs_atp3.dta")

# NHANES Continuous

## Process

In [202]:
nhanes1999_2000 = pd.read_stata("/Users/bryankim/Documents/NBER/Case Deaton/Data/FRS/ATP3/all_nhanes1999_2000_for_frs_atp3.dta")
#nhanes1999_2000=nhanes1999_2000.dropna(subset=["sex","age","hdl","lbxtc","asbp","smq020","treated_hpt"])

In [203]:
nhanes1999_2000[["riagendr","age","hdl","lbxtc","asbp","smq020","treated_hpt","treated_hchol","treated_diabetes"]].head()
#list(nhanes1999_2000.columns) 

,riagendr,age,hdl,lbxtc,asbp,smq020,treated_hpt,treated_hchol,treated_diabetes
0,1.0,49.0,42.0,279.0,122.0,1.0,1.0,0.0,0.0
1,2.0,59.0,105.0,245.0,123.0,1.0,0.0,0.0,0.0
2,1.0,70.0,49.0,314.0,130.0,1.0,1.0,0.0,1.0
3,2.0,53.0,105.0,219.0,114.0,1.0,0.0,0.0,0.0
4,1.0,62.0,49.0,216.0,124.0,1.0,1.0,0.0,1.0


In [199]:
nhanes1999_2000.age.head()

0    49.0
1    59.0
2    70.0
3    53.0
4    62.0
Name: age, dtype: float32

In [ ]:
/Users/bryankim/Documents/NBER/Case Deaton/Data/FRS/ATP3/nhanes_
/Users/bryankim/Documents/NBER/Case Deaton/Data/FRS/ATP3/all_nhanes

In [200]:
nhanes1999_2000 = pd.read_stata("/Users/bryankim/Documents/NBER/Case Deaton/Data/FRS/ATP3/all_nhanes1999_2000_for_frs_atp3.dta")
nhanes2001_2002 = pd.read_stata("/Users/bryankim/Documents/NBER/Case Deaton/Data/FRS/ATP3/all_nhanes1999_2000_for_frs_atp3.dta")
nhanes2003_2004 = pd.read_stata("/Users/bryankim/Documents/NBER/Case Deaton/Data/FRS/ATP3/all_nhanes2003_2004_for_frs_atp3.dta")

nhanes2005_2006 = pd.read_stata("/Users/bryankim/Documents/NBER/Case Deaton/Data/FRS/ATP3/all_nhanes2005_2006_for_frs_atp3.dta")

nhanes2007_2008 = pd.read_stata("/Users/bryankim/Documents/NBER/Case Deaton/Data/FRS/ATP3/all_nhanes2007_2008_for_frs_atp3.dta")

nhanes2009_2010 = pd.read_stata("/Users/bryankim/Documents/NBER/Case Deaton/Data/FRS/ATP3/all_nhanes2009_2010_for_frs_atp3.dta")

nhanes2011_2012 = pd.read_stata("/Users/bryankim/Documents/NBER/Case Deaton/Data/FRS/ATP3/all_nhanes2011_2012_for_frs_atp3.dta")

nhanes2013_2014 = pd.read_stata("/Users/bryankim/Documents/NBER/Case Deaton/Data/FRS/ATP3/all_nhanes2013_2014_for_frs_atp3.dta")

nhanes2015_2016 = pd.read_stata("/Users/bryankim/Documents/NBER/Case Deaton/Data/FRS/ATP3/all_nhanes2015_2016_for_frs_atp3.dta")

nhanes2017_2018 = pd.read_stata("/Users/bryankim/Documents/NBER/Case Deaton/Data/FRS/ATP3/all_nhanes2017_2018_for_frs_atp3.dta")

nhanes1999_2000.sex.count()

2091

In [204]:
nhanes1999_2000=nhanes1999_2000.dropna(subset=["riagendr","age","hdl","lbxtc","asbp","smq020","treated_hpt"])
nhanes2001_2002=nhanes2001_2002.dropna(subset=["riagendr","age","hdl","lbxtc","asbp","smq020","treated_hpt"])
nhanes2003_2004=nhanes2003_2004.dropna(subset=["riagendr","age","hdl","lbxtc","asbp","smq020","treated_hpt"])
nhanes2005_2006=nhanes2005_2006.dropna(subset=["riagendr","age","hdl","lbxtc","asbp","smq020","treated_hpt"])
nhanes2007_2008=nhanes2007_2008.dropna(subset=["riagendr","age","hdl","lbxtc","asbp","smq020","treated_hpt"])
nhanes2009_2010=nhanes2009_2010.dropna(subset=["riagendr","age","hdl","lbxtc","asbp","smq020","treated_hpt"])
nhanes2011_2012=nhanes2011_2012.dropna(subset=["riagendr","age","hdl","lbxtc","asbp","smq020","treated_hpt"])
nhanes2013_2014=nhanes2013_2014.dropna(subset=["riagendr","age","hdl","lbxtc","asbp","smq020","treated_hpt"])
nhanes2015_2016=nhanes2015_2016.dropna(subset=["riagendr","age","hdl","lbxtc","asbp","smq020","treated_hpt"])
nhanes2017_2018=nhanes2017_2018.dropna(subset=["riagendr","age","hdl","lbxtc","asbp","smq020","treated_hpt"])

# count number of rows
nhanes1999_2000.riagendr.count()


1754

In [205]:
def get_frs_atp3_cont(x):
    risk_score = 0
    if x["riagendr"] == 1:
        risk_score += mage_pts(x["age"])
        risk_score += mhdl_pts(x["hdl"])
        risk_score += mtchol_pts(x["lbxtc"],x["age"])
        risk_score += msbp_pts(x["asbp"],x["treated_hpt"])
        risk_score += msmoke_pts(x["curr_smq"],x["age"])
    if x["riagendr"] == 2:
        risk_score += fage_pts(x["age"])
        risk_score += fhdl_pts(x["hdl"])
        risk_score += ftchol_pts(x["lbxtc"],x["age"])
        risk_score += fsbp_pts(x["asbp"],x["treated_hpt"])
        risk_score += fsmoke_pts(x["curr_smq"],x["age"])
    return(risk_score)


In [206]:
nhanes1999_2000["frs_atp3"]=nhanes1999_2000.apply(get_frs_atp3_cont, axis=1)
nhanes2001_2002["frs_atp3"]=nhanes2001_2002.apply(get_frs_atp3_cont, axis=1)
nhanes2003_2004["frs_atp3"]=nhanes2003_2004.apply(get_frs_atp3_cont, axis=1)
nhanes2005_2006["frs_atp3"]=nhanes2005_2006.apply(get_frs_atp3_cont, axis=1)
nhanes2007_2008["frs_atp3"]=nhanes2007_2008.apply(get_frs_atp3_cont, axis=1)
nhanes2009_2010["frs_atp3"]=nhanes2009_2010.apply(get_frs_atp3_cont, axis=1)
nhanes2011_2012["frs_atp3"]=nhanes2011_2012.apply(get_frs_atp3_cont, axis=1)
nhanes2013_2014["frs_atp3"]=nhanes2013_2014.apply(get_frs_atp3_cont, axis=1)
nhanes2015_2016["frs_atp3"]=nhanes2015_2016.apply(get_frs_atp3_cont, axis=1)
nhanes2017_2018["frs_atp3"]=nhanes2017_2018.apply(get_frs_atp3_cont, axis=1)

In [207]:
nhanes1999_2000["frs_atp3"].head()

0    11
1    13
2    16
3    13
4    13
Name: frs_atp3, dtype: int64

In [209]:
nhanes1999_2000.to_stata("/Users/bryankim/Documents/NBER/Case Deaton/Data/FRS/ATP3/for FRS/nhanes_1999_2000_frs_atp3.dta")
nhanes2001_2002.to_stata("/Users/bryankim/Documents/NBER/Case Deaton/Data/FRS/ATP3/for FRS/nhanes_2001_2002_frs_atp3.dta")
nhanes2003_2004.to_stata("/Users/bryankim/Documents/NBER/Case Deaton/Data/FRS/ATP3/for FRS/nhanes_2003_2004_frs_atp3.dta")
nhanes2005_2006.to_stata("/Users/bryankim/Documents/NBER/Case Deaton/Data/FRS/ATP3/for FRS/nhanes_2005_2006_frs_atp3.dta")
nhanes2007_2008.to_stata("/Users/bryankim/Documents/NBER/Case Deaton/Data/FRS/ATP3/for FRS/nhanes_2007_2008_frs_atp3.dta")
nhanes2009_2010.to_stata("/Users/bryankim/Documents/NBER/Case Deaton/Data/FRS/ATP3/for FRS/nhanes_2009_2010_frs_atp3.dta")
nhanes2011_2012.to_stata("/Users/bryankim/Documents/NBER/Case Deaton/Data/FRS/ATP3/for FRS/nhanes_2011_2012_frs_atp3.dta")
nhanes2013_2014.to_stata("/Users/bryankim/Documents/NBER/Case Deaton/Data/FRS/ATP3/for FRS/nhanes_2013_2014_frs_atp3.dta")
nhanes2015_2016.to_stata("/Users/bryankim/Documents/NBER/Case Deaton/Data/FRS/ATP3/for FRS/nhanes_2015_2016_frs_atp3.dta")
nhanes2017_2018.to_stata("/Users/bryankim/Documents/NBER/Case Deaton/Data/FRS/ATP3/for FRS/nhanes_2017_2018_frs_atp3.dta")

In [213]:
list(nhanes2011_2012.columns)

['seqn',
 'sddsrvyr',
 'ridstatr',
 'ridexmon',
 'riagendr',
 'ridageyr',
 'ridagemn',
 'ridageex',
 'ridreth1',
 'ridreth2',
 'dmqmilit',
 'dmdborn',
 'dmdcitzn',
 'dmdyrsus',
 'dmdeduc3',
 'grade',
 'dmdeduc',
 'dmdschol',
 'dmdmartl',
 'dmdhhsiz',
 'indhhinc',
 'indfminc',
 'indfmpir',
 'ridexprg',
 'ridpreg',
 'dmdhrgnd',
 'dmdhrage',
 'dmdhrbrn',
 'dmdhredu',
 'dmdhrmar',
 'dmdhsedu',
 'wtint2yr',
 'wtint4yr',
 'wtmec2yr',
 'wtmec4yr',
 'sdmvpsu',
 'sdmvstra',
 'sdj1repn',
 'dmaethn',
 'dmarace',
 'wtmrep01',
 'wtmrep02',
 'wtmrep03',
 'wtmrep04',
 'wtmrep05',
 'wtmrep06',
 'wtmrep07',
 'wtmrep08',
 'wtmrep09',
 'wtmrep10',
 'wtmrep11',
 'wtmrep12',
 'wtmrep13',
 'wtmrep14',
 'wtmrep15',
 'wtmrep16',
 'wtmrep17',
 'wtmrep18',
 'wtmrep19',
 'wtmrep20',
 'wtmrep21',
 'wtmrep22',
 'wtmrep23',
 'wtmrep24',
 'wtmrep25',
 'wtmrep26',
 'wtmrep27',
 'wtmrep28',
 'wtmrep29',
 'wtmrep30',
 'wtmrep31',
 'wtmrep32',
 'wtmrep33',
 'wtmrep34',
 'wtmrep35',
 'wtmrep36',
 'wtmrep37',
 'wtmrep38',